In [ ]:
import pandas as pd
from mainnet_launch.constants import *
import requests
import plotly.express as px

import os
from mainnet_launch.data_fetching.tokemak_subgraph import run_query_with_paginate


from mainnet_launch.data_fetching.get_state_by_block import *


autopool_address_to_name = {a.autopool_eth_addr: a.name for a in ALL_AUTOPOOLS}


def get_rebalance_volumne_raw_data(chain: ChainData):
    query = """
    query getAutopoolRebalances($first: Int!, $skip: Int!) {
      autopoolRebalances(
        first: $first,
        skip: $skip,
        orderBy: timestamp,
        orderDirection: asc,
      ) {
        autopool
        destinationInName
        destinationOutName
        timestamp
        blockNumber
        transactionHash
        tokenOutValueInEth
        tokenOutValueBaseAsset
      }
    }
    """

    if chain.chain_id == 1:
        api_url = os.environ["TOKEMAK_ETHEREUM_SUBGRAPH_URL"]
    elif chain.chain_id == 8453:
        api_url = os.environ["TOKEMAK_BASE_SUBGRAPH_URL"]
    elif chain.chain_id == 146:
        api_url = os.environ["TOKEMAK_SONIC_SUBGRAPH_URL"]

    variables = {}
    df = run_query_with_paginate(api_url, query, variables, "autopoolRebalances")
    df["chain_id"] = chain.chain_id
    df["autopool_name"] = df["autopool"].apply(
        lambda x: autopool_address_to_name[ETH_CHAIN.client.toChecksumAddress(x)]
    )
    df["datetime"] = pd.to_datetime(df["timestamp"].astype(int), unit="s", utc=True)
    df["date"] = df["datetime"].dt.date

    df["block"] = df["blockNumber"].astype(int)
    df["tokenOutValueInEth_norm"] = df["tokenOutValueInEth"].apply(lambda x: int(x) / 1e18)
    return df


def get_sonic_rebalance_volumne_df():

    df = get_rebalance_volumne_raw_data(SONIC_CHAIN)

    if np.any(df["autopool_name"] != "sonicUSD"):
        raise ValueError("Need custom logic to get the Sonic `S` price to USDC")

    df["computed_usd_volumne"] = df["tokenOutValueBaseAsset"].apply(lambda x: int(x) / 1e6)

    return df.set_index("datetime").sort_index()


def get_mainnet_rebalance_volumne_df():
    df = get_rebalance_volumne_raw_data(ETH_CHAIN)

    calls = [
        Call(
            ROOT_PRICE_ORACLE(ETH_CHAIN),
            ["getPriceInQuote(address,address)(uint256)", WETH(ETH_CHAIN), USDC(ETH_CHAIN)],
            [("mainnet_ETH_TO_USDC_price", safe_normalize_6_with_bool_success)],
        )
    ]

    blocks = df["blockNumber"].astype(int)
    # slow
    print(len(blocks), ETH_CHAIN.name)
    base_weth_price_df = get_raw_state_by_blocks(calls, blocks, ETH_CHAIN, include_block_number=True)
    full_df = pd.merge(df, base_weth_price_df, on="block")

    full_df["computed_usd_volumne"] = full_df["tokenOutValueInEth_norm"] * full_df["mainnet_ETH_TO_USDC_price"]
    return full_df.set_index("datetime").sort_index()


def get_base_rebalance_volume_df():
    df = get_rebalance_volumne_raw_data(BASE_CHAIN)

    calls = [
        Call(
            ROOT_PRICE_ORACLE(BASE_CHAIN),
            ["getPriceInQuote(address,address)(uint256)", WETH(BASE_CHAIN), USDC(BASE_CHAIN)],
            [("base_chain_ETH_to_USDC_price", safe_normalize_6_with_bool_success)],
        )
    ]

    blocks = df["blockNumber"].astype(int)

    base_weth_price_df = get_raw_state_by_blocks(calls, blocks, BASE_CHAIN, include_block_number=True)
    full_df = pd.merge(df, base_weth_price_df, on="block")

    full_df["computed_usd_volumne"] = full_df["tokenOutValueInEth_norm"] * full_df["base_chain_ETH_to_USDC_price"]
    return full_df.set_index("datetime").sort_index()


base_df = get_base_rebalance_volume_df()
mainnet_df = get_mainnet_rebalance_volumne_df()
sonic_df = get_sonic_rebalance_volumne_df()

# 3:20

1723 eth


In [92]:
df = pd.concat([mainnet_df, sonic_df, base_df], axis=0).reset_index()
df

,datetime,autopool,destinationInName,destinationOutName,timestamp,blockNumber,transactionHash,tokenOutValueInEth,tokenOutValueBaseAsset,chain_id,autopool_name,date,block,tokenOutValueInEth_norm,mainnet_ETH_TO_USDC_price,computed_usd_volumne,base_chain_ETH_to_USDC_price
0,2024-09-16 20:41:35+00:00,0x6dc3ce9c57b20131347fdc9089d740daf6eb34c5,Tokemak-Wrapped Ether-Balancer rsETH / ETHx,idle,1726519295,20765628,0xedaf8a039095acfb8b6f7e449b44f417d3a110766d1e...,121714202617450528768,121714202617450528768,1,balETH,2024-09-16,20765628,121.714203,2292.301285,279005.623063,NaN
1,2024-09-17 02:30:23+00:00,0xe800e3760fc20aa98c5df6a9816147f190455af3,Tokemak-Wrapped Ether-weETH/WETH-ng,idle,1726540223,20767364,0x5a6f3a2239134ef903468d88abce973e6b71e233af04...,103024423626862362624,103024423626862362624,1,autoLRT,2024-09-17,20767364,103.024424,2268.772316,233738.960196,NaN
2,2024-09-17 08:02:23+00:00,0x6dc3ce9c57b20131347fdc9089d740daf6eb34c5,Tokemak-Wrapped Ether-Gyroscope ECLP wstETH/wETH,idle,1726560143,20769006,0x0f300d5fb1bfc46d634a9b64f773dbfd73ab9babafc1...,135862072073420324864,135862072073420324864,1,balETH,2024-09-17,20769006,135.862072,2291.717100,311357.433812,NaN
3,2024-09-17 18:34:35+00:00,0xe800e3760fc20aa98c5df6a9816147f190455af3,Tokemak-Wrapped Ether-Balancer rsETH-WETH Stab...,idle,1726598075,20772145,0x46cfba2d4f806b97e71d9465ae05556bc41cb3635c5d...,105124100397257129984,105124100397257129984,1,autoLRT,2024-09-17,20772145,105.124100,2369.677038,249110.166852,NaN
4,2024-09-17 19:42:23+00:00,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56,Tokemak-Wrapped Ether-pxETH/wETH,idle,1726602143,20772483,0xf4b0efb2e5d69ef7b7d000b4860a9d86b76013d9a042...,100000000000000000000,100000000000000000000,1,autoETH,2024-09-17,20772483,100.000000,2345.341416,234534.141600,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2553,2025-07-01 11:57:41+00:00,0x9c6864105aec23388c89600046213a44c384c831,Tokemak-USD Coin-Balancer Aave USDC-Aave GHO,idle,1751371061,32290857,0x8d6114d6d751515841ae119fe03ae1f7645992a85301...,9191749208952607112,22514419700,8453,baseUSD,2025-07-01,32290857,9.191749,NaN,22514.419694,2449.416230
2554,2025-07-01 13:57:47+00:00,0x9c6864105aec23388c89600046213a44c384c831,Tokemak-USD Coin-Balancer Aave USDC-Aave GHO,idle,1751378267,32294460,0xfa0e7c882c5660a45e0b004166adfb5024496393cb37...,7361090804983176000,18000000000,8453,baseUSD,2025-07-01,32294460,7.361091,NaN,17999.999997,2445.289764
2555,2025-07-01 15:57:57+00:00,0x9c6864105aec23388c89600046213a44c384c831,Tokemak-USD Coin-Balancer Aave USDC-Aave GHO,idle,1751385477,32298065,0xc421176d59e3955d45d9a698b375425601fed55a413e...,7426603721172690000,18000000000,8453,baseUSD,2025-07-01,32298065,7.426604,NaN,17999.999995,2423.718926
2556,2025-07-01 17:57:55+00:00,0x9c6864105aec23388c89600046213a44c384c831,Tokemak-USD Coin-Balancer Aave USDC-Aave GHO,idle,1751392675,32301664,0x42c2f51f067c9a9707a7183c6dc226d8fec952c3e708...,7414194090532734000,18000000000,8453,baseUSD,2025-07-01,32301664,7.414194,NaN,17999.999996,2427.775666


In [102]:
import plotly.io as pio
import plotly.express as px

# to disable auto‐rendering entirely:
pio.renderers.default = "notebook_connected"
pio.templates.default = None

volume_by_pool = df.groupby(["date", "autopool_name"])["computed_usd_volumne"].sum().reset_index()

# 2. Pivot so each autopool is its own column
pivoted = volume_by_pool.pivot(index="date", columns="autopool_name", values="computed_usd_volumne").fillna(
    0
)  # fill days with 0 rebalances with 0s         # fill missing dates/pools with zero

cum_volume = pivoted.sort_index().cumsum().ffill()
cum_volume.index = pd.to_datetime(cum_volume.index)
cum_volume = cum_volume.resample('D').first()
# 4. Plot
fig = px.bar(
    cum_volume,
    title="Cumulative USD Volume by Autopool",
    labels={"value": "Cumulative USD Volume", "date": "Date", "autopool_name": "Autopool"},
)
fig.show()